In [1]:
import json
from os import path, listdir
import gzip
import pandas as pd

In [2]:
datasets_dir = path.join(path.curdir, '..', 'datasets')

In [3]:
filenames = listdir(datasets_dir)
gzip_games_filepath = path.join(datasets_dir, filenames[0])
gzip_movies_filepath = path.join(datasets_dir, filenames[1])

In [4]:
def read_json_gz(gzip_filepath):
    decompressed = gzip.open(gzip_filepath)
    lines = decompressed.readlines()
    data = [json.loads(line) for line in lines]
    return data

In [5]:
data_games = read_json_gz(gzip_games_filepath)

In [6]:
data_movies = read_json_gz(gzip_movies_filepath)

In [7]:
import nltk
from nltk.corpus import stopwords

In [8]:
game_reviews = list(map(lambda obj: obj['reviewText'], data_games))
movie_reviews = list(map(lambda obj: obj['reviewText'], data_movies))

In [9]:
tokenized_games_data = [nltk.wordpunct_tokenize(review) for review in game_reviews[:1000]]
tokenized_movies_data = [nltk.wordpunct_tokenize(review) for review in movie_reviews[:1000]]

In [10]:
tokens = set([inner for outer in tokenized_games_data + tokenized_movies_data for inner in outer])

In [11]:
game_review_matrix = []
for review in tokenized_games_data:
    data = [review.count(token) for token in tokens]
    game_review_matrix.append(data)

In [12]:
movie_review_matrix = []
for review in tokenized_movies_data:
    data = [review.count(token) for token in tokens]
    movie_review_matrix.append(data)

In [13]:
X = game_review_matrix + movie_review_matrix
Y = [1 for i in range(0, len(game_review_matrix))] + [-1 for i in range(0, len(movie_review_matrix))]

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [17]:
nb = GaussianNB()

In [18]:
nb.fit(X_train, y_train)

GaussianNB(priors=None)

In [19]:
y_train_predict = nb.predict(X_train)
y_test_predict = nb.predict(X_test)

err_train = np.mean(y_train != y_train_predict)
err_test  = np.mean(y_test  != y_test_predict)

print(err_train, err_test)

0.01625 0.0475


In [ ]:
## заюзать разреженные матрицы